In [1]:
import  pandas  as  pd
import numpy as np
import os
import requests
import matplotlib.pyplot as plt
import geopandas as gpd
import pyproj
import datetime
from geopandas import GeoDataFrame as gdf
from shapely.geometry import Point
from shapely.geometry import Polygon
import json
import sys
sys.path.append('/Users/alenastern/Documents/Spring2018/Machine_Learning/evictions-learn/src/data')
import scraper as sp
%matplotlib inline

# Querying Census API

In [2]:
state_list = ['18', '17', '26', '39', '55', '10', '11', '12', '13', '24', '37', '45', '51', '54']
df = pd.read_table('national_county.txt', sep = ',', converters={'01': lambda x: str(x), '001': lambda x: str(x)})
df = df[df['01'].isin(state_list)]

In [3]:
df['county'] = df['01'] + df['001']

In [4]:
len(df)

1026

In [5]:
acs_results_2016 = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B15002_001E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B25010_001E&for=block%20group:*&in=state:10%20county:001&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')
data = acs_results_2016.text
total_df = pd.read_json(data)
errors = []


# Query ACS Data from Census API
for i in range(1, len(df)):
    acs_results_2016 = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B15002_001E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B25010_001E&for=block%20group:*&in=state:{}%20county:{}&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4'.format(df.iloc[i,1], df.iloc[i,2]))
    print(df.iloc[i,1], df.iloc[i,2])
    if acs_results_2016.status_code != 204:
        data = acs_results_2016.text
        acs_df = pd.read_json(data)
        total_df = pd.concat([total_df, acs_df])
    else:
        errors.append([df.iloc[i,1], df.iloc[i,2]])



10 003
10 005
11 001
12 001
12 003
12 005
12 007
12 009
12 011
12 013
12 015
12 017
12 019
12 021
12 023
12 027
12 029
12 031
12 033
12 035
12 037
12 039
12 041
12 043
12 045
12 047
12 049
12 051
12 053
12 055
12 057
12 059
12 061
12 063
12 065
12 067
12 069
12 071
12 073
12 075
12 077
12 079
12 081
12 083
12 085
12 086
12 087
12 089
12 091
12 093
12 095
12 097
12 099
12 101
12 103
12 105
12 107
12 109
12 111
12 113
12 115
12 117
12 119
12 121
12 123
12 125
12 127
12 129
12 131
12 133
13 001
13 003
13 005
13 007
13 009
13 011
13 013
13 015
13 017
13 019
13 021
13 023
13 025
13 027
13 029
13 031
13 033
13 035
13 037
13 039
13 043
13 045
13 047
13 049
13 051
13 053
13 055
13 057
13 059
13 061
13 063
13 065
13 067
13 069
13 071
13 073
13 075
13 077
13 079
13 081
13 083
13 085
13 087
13 089
13 091
13 093
13 095
13 097
13 099
13 101
13 103
13 105
13 107
13 109
13 111
13 113
13 115
13 117
13 119
13 121
13 123
13 125
13 127
13 129
13 131
13 133
13 135
13 137
13 139
13 141
13 143
13 145
13 147

KeyboardInterrupt: 

In [ ]:
# Clean ACS data frame and rename columns# Clean  
variables = {'B15002_001E': "total_pop_over25", 'B15003_022E': "bachelors", 'B15003_023E': "masters", 'B15003_024E': "professional", 'B15003_024E': "doctorate"
            'B25010_001E': "avg_hh_size"}
acs_df.columns = acs_df.iloc[0]
acs_df.drop([0], inplace = True)
acs_df.rename(columns = variables, inplace=True)

# Scraping Building Permit Data

In [ ]:
ul = sp.scrape_links('https://www2.census.gov/econ/bps/County/', [])

In [ ]:
permits = sp.url_to_pd(ul)

In [ ]:
permits.columns

In [ ]:
permits.iloc[4]

In [ ]:
permits = permits.reset_index()

In [ ]:
permits.shape

In [ ]:
permits = permits[permits.duplicated() == False]

In [ ]:
permits.head()

In [ ]:
permits.rename(index=str, columns={"index": "year", "Survey": "state", "FIPS": "county", "FIPS.1": "region",
                             "Region": "division", "Division": "name", "County": "bldg_1", "Unnamed: 6": "units_1",
                             "1-unit": "value_1", "Unnamed: 8": "bldg_2", "Unnamed: 9": "units_2", "2-units": "value_2",
                              "Unnamed: 11": "bldg_3", "Unnamed: 12": "units_3", "3-4 units": "value_3-4",
                              "Unnamed: 14": "bldg_5", "Unnamed: 15": "units_5", "5+ units": "value_5"}, inplace = True)

In [ ]:
permits.drop(['Unnamed: 17', 'Unnamed: 18', '1-unit rep', 'Unnamed: 20',
       'Unnamed: 21', '2-units rep', 'Unnamed: 23', 'Unnamed: 24',
       '3-4 units rep', 'Unnamed: 26', 'Unnamed: 27', ' 5+units rep'], axis=1, inplace = True)

In [ ]:
permits.drop('0', axis = 0, inplace = True)

In [ ]:
permits.head()

In [ ]:
cols = ['bldg_1', 'units_1', 'value_1', 'bldg_2', 'units_2', 'value_2', 'bldg_3', 'units_3', 'value_3-4',
       'bldg_5', 'units_5', 'value_5']

permits[cols] = permits[cols].apply(pd.to_numeric, errors='coerce')


permits['total_bldg'] = permits['bldg_1'] + permits['bldg_2'] + permits['bldg_3'] + permits['bldg_5'] 
permits['total_units'] = permits['units_1'] + permits['units_2'] + permits['units_3'] + permits['units_5']
permits['total_value'] = permits['value_1'] + permits['value_2'] + permits['value_3-4'] + permits['value_5']

In [ ]:
permits_div = permits[(permits['division'] == '3') | (permits['division'] == '5')]

In [ ]:
permits_div.iloc[4]

In [ ]:
permits_total = permits_div[['year', 'state', 'county', 'region', 'division', 'total_bldg', 'total_units', 'total_value']]

In [ ]:
permits_total.head()

In [ ]:
permits_total['geo_id'] = permits['state'] + permits['county']

In [ ]:
permits_total.total_units = permits_total.total_units.astype(int)
permits_total.total_bldg = permits_total.total_bldg.astype(int)

In [ ]:
permits_total.head()

In [ ]:
permits_total = permits_total[permits_total.duplicated()==False]

In [ ]:
permits_total.to_csv('permits.csv', index = False)

# Household Size Data

In [3]:
hs_census = pd.read_csv('raw/census/hhsize.csv')

In [ ]:
len(hs_census.iloc[0,0])

In [4]:
hs_census = hs_census[(hs_census['STATEA'] == 18) | (hs_census['STATEA'] == 17) | (hs_census['STATEA'] == 26) |
                     (hs_census['STATEA'] == 39) |
                     (hs_census['STATEA'] == 55) |
                     (hs_census['STATEA'] == 10) |
                     (hs_census['STATEA'] == 11) |
                     (hs_census['STATEA'] == 12) |
                     (hs_census['STATEA'] == 13) |
                     (hs_census['STATEA'] == 24) |
                     (hs_census['STATEA'] == 37) |
                     (hs_census['STATEA'] == 45) |
                     (hs_census['STATEA'] == 51) |
                     (hs_census['STATEA'] == 54)]

In [6]:
hs_census = hs_census[(hs_census['DATAYEAR'] == 2000) | (hs_census['DATAYEAR'] == 2010) ]

In [7]:
hs_census.rename(index = str, columns = {'DATAYEAR' : 'Year', 'CM4AA' : 'Total_HH', 'CS2AA': 'HHF_2', 'CS2AB': 'HHF_3', 'CS2AC': 'HHF_4',
          'CS2AD' : 'HHF_5', 'CS2AE' : 'HHF_6', 'CS2AF': 'HHF_7', 'CS2AG' : 'HHN_1',
          'CS2AH' : 'HHN_2', 'CS2AI' : 'HHN_3', 'CS2AJ': 'HHN_4', 'CS2AK': 'HHN_5',
          'CS2AL' : 'HHN_6', 'CS2AM' :'HHN_7'}, inplace = True)

In [8]:
hs_census['geo_id'] = hs_census.GISJOIN.str[1:3] + hs_census.GISJOIN.str[4:7] + hs_census.GISJOIN.str[8:15] 

In [9]:
hs_census = hs_census[['Year','Total_HH', 'HHF_2', 'HHF_3', 'HHF_4', 'HHF_5', 'HHF_6', 'HHF_7', 'HHN_1', 'HHN_2',
                      'HHN_3', 'HHN_4', 'HHN_5', 'HHN_6', 'HHN_7', 'geo_id']]

In [10]:
hs_census['total_ppl'] = hs_census['HHF_2']*2 + hs_census['HHF_3']*3 + hs_census['HHF_4']*4 + hs_census['HHF_5']*5 + hs_census['HHF_6']*6 + hs_census['HHF_7']*7 + hs_census['HHN_1'] + hs_census['HHN_2']*2 + hs_census['HHN_3']*3 + hs_census['HHN_4']*4 + hs_census['HHN_5']*5 + hs_census['HHN_6']*6 + hs_census['HHN_7']*7

In [11]:
hs_census.head()

,Year,Total_HH,HHF_2,HHF_3,HHF_4,HHF_5,HHF_6,HHF_7,HHN_1,HHN_2,HHN_3,HHN_4,HHN_5,HHN_6,HHN_7,geo_id,total_ppl
257366,2000,662.0,201.0,123.0,144.0,46.0,18.0,15.0,88.0,20.0,4.0,3.0,0.0,0.0,0.0,100010401001,1942.0
257367,2000,593.0,182.0,108.0,124.0,39.0,10.0,13.0,87.0,22.0,5.0,3.0,0.0,0.0,0.0,100010401002,1688.0
257368,2000,595.0,173.0,105.0,83.0,35.0,20.0,26.0,119.0,32.0,1.0,1.0,0.0,0.0,0.0,100010401003,1660.0
257369,2000,623.0,183.0,93.0,105.0,42.0,11.0,7.0,151.0,23.0,2.0,4.0,1.0,1.0,0.0,100010402011,1620.0
257370,2000,749.0,174.0,131.0,87.0,47.0,23.0,6.0,224.0,51.0,4.0,0.0,2.0,0.0,0.0,100010402012,1852.0


In [12]:
hs_census['avg_size'] = hs_census['total_ppl']/ hs_census['Total_HH']

In [16]:
hs_census_final = hs_census[['Year', 'geo_id', 'avg_size']]

In [19]:
hs_census10 = pd.read_csv('raw/census/hhsize_acs_2010.csv')

In [16]:
hs_census10.columns

Index(['GISJOIN', 'YEAR', 'REGIONA', 'DIVISIONA', 'STATE', 'STATEA', 'COUNTY',
       'COUNTYA', 'COUSUBA', 'PLACEA', 'TRACTA', 'BLKGRPA', 'CONCITA',
       'AIANHHA', 'RES_ONLYA', 'TRUSTA', 'AITSCEA', 'ANRCA', 'CBSAA', 'CSAA',
       'METDIVA', 'NECTAA', 'CNECTAA', 'NECTADIVA', 'UAA', 'CDCURRA', 'SLDUA',
       'SLDLA', 'SUBMCDA', 'SDELMA', 'SDSECA', 'SDUNIA', 'PUMA5A', 'BTTRA',
       'BTBGA', 'NAME_E', 'JNWE001', 'JNWE002', 'JNWE003', 'JNWE004',
       'JNWE005', 'JNWE006', 'JNWE007', 'JNWE008', 'JNWE009', 'JNWE010',
       'JNWE011', 'JNWE012', 'JNWE013', 'JNWE014', 'JNWE015', 'JNWE016',
       'NAME_M', 'JNWM001', 'JNWM002', 'JNWM003', 'JNWM004', 'JNWM005',
       'JNWM006', 'JNWM007', 'JNWM008', 'JNWM009', 'JNWM010', 'JNWM011',
       'JNWM012', 'JNWM013', 'JNWM014', 'JNWM015', 'JNWM016'],
      dtype='object')

In [20]:
hs_census10.rename(index = str, columns = {'YEAR' : 'Year', 'JNWE001' : 'Total_HH', 'JNWE003': 'HHF_2', 'JNWE004': 'HHF_3', 'JNWE005': 'HHF_4',
          'JNWE006' : 'HHF_5', 'JNWE007' : 'HHF_6', 'JNWE008': 'HHF_7', 'JNWE010' : 'HHN_1',
          'JNWE011' : 'HHN_2', 'JNWE012' : 'HHN_3', 'JNWE013': 'HHN_4', 'JNWE014': 'HHN_5',
          'JNWE015' : 'HHN_6', 'JNWE016' :'HHN_7'}, inplace = True)
hs_census10['geo_id'] = hs_census10.GISJOIN.str[1:3] + hs_census10.GISJOIN.str[4:7] + hs_census10.GISJOIN.str[8:15]
hs_census10 = hs_census10[['Year','Total_HH', 'HHF_2', 'HHF_3', 'HHF_4', 'HHF_5', 'HHF_6', 'HHF_7', 'HHN_1', 'HHN_2',
                      'HHN_3', 'HHN_4', 'HHN_5', 'HHN_6', 'HHN_7', 'geo_id']]
hs_census10['total_ppl'] = hs_census10['HHF_2']*2 + hs_census10['HHF_3']*3 + hs_census10['HHF_4']*4 + hs_census10['HHF_5']*5 + hs_census10['HHF_6']*6 + hs_census10['HHF_7']*7 + hs_census10['HHN_1'] + hs_census10['HHN_2']*2 + hs_census10['HHN_3']*3 + hs_census10['HHN_4']*4 + hs_census10['HHN_5']*5 + hs_census10['HHN_6']*6 + hs_census10['HHN_7']*7
hs_census10['avg_size'] = hs_census10['total_ppl']/ hs_census10['Total_HH']
hs_census_final10 = hs_census10[['Year', 'geo_id', 'avg_size']]

In [11]:
hs_census10['geo_id'] = hs_census10.GISJOIN.str[1:3] + hs_census10.GISJOIN.str[4:7] + hs_census10.GISJOIN.str[8:15] 

In [12]:
hs_census10 = hs_census10[['Year','Total_HH', 'HHF_2', 'HHF_3', 'HHF_4', 'HHF_5', 'HHF_6', 'HHF_7', 'HHN_1', 'HHN_2',
                      'HHN_3', 'HHN_4', 'HHN_5', 'HHN_6', 'HHN_7', 'geo_id']]

In [13]:
hs_census10['total_ppl'] = hs_census10['HHF_2']*2 + hs_census10['HHF_3']*3 + hs_census10['HHF_4']*4 + hs_census10['HHF_5']*5 + hs_census10['HHF_6']*6 + hs_census10['HHF_7']*7 + hs_census10['HHN_1'] + hs_census10['HHN_2']*2 + hs_census10['HHN_3']*3 + hs_census10['HHN_4']*4 + hs_census10['HHN_5']*5 + hs_census10['HHN_6']*6 + hs_census10['HHN_7']*7

In [21]:
hs_census10['avg_size'] = hs_census10['total_ppl']/ hs_census10['Total_HH']

In [23]:
hs_census_final10 = hs_census10[['Year', 'geo_id', 'avg_size']]

In [21]:
hs_census15 = pd.read_csv('raw/census/hhsize_acs_2015.csv')
hs_census15.head()

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,...,ADMKM007,ADMKM008,ADMKM009,ADMKM010,ADMKM011,ADMKM012,ADMKM013,ADMKM014,ADMKM015,ADMKM016
0,G10000100401001,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,45,11,115,116,27,11,11,11,11,11
1,G10000100401002,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,26,14,83,81,14,11,11,11,11,11
2,G10000100401003,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,23,11,90,45,76,21,11,11,11,11
3,G10000100402011,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,28,11,46,44,17,11,11,11,11,11
4,G10000100402012,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,20,19,130,123,47,11,11,11,11,11


In [22]:
# households 7 or above coded as 7 people 
hs_census15.rename(index = str, columns = {'YEAR' : 'Year', 'ADMKE001' : 'Total_HH', 'ADMKE003': 'HHF_2', 'ADMKE004': 'HHF_3', 'ADMKE005': 'HHF_4',
          'ADMKE006' : 'HHF_5', 'ADMKE007' : 'HHF_6', 'ADMKE008': 'HHF_7', 'ADMKE010' : 'HHN_1',
          'ADMKE011' : 'HHN_2', 'ADMKE012' : 'HHN_3', 'ADMKE013': 'HHN_4', 'ADMKE014': 'HHN_5',
          'ADMKE015' : 'HHN_6', 'ADMKE016' :'HHN_7'}, inplace = True)
hs_census15['geo_id'] = hs_census15.GISJOIN.str[1:3] + hs_census15.GISJOIN.str[4:7] + hs_census15.GISJOIN.str[8:15] 
hs_census15 = hs_census15[['Year','Total_HH', 'HHF_2', 'HHF_3', 'HHF_4', 'HHF_5', 'HHF_6', 'HHF_7', 'HHN_1', 'HHN_2',
                      'HHN_3', 'HHN_4', 'HHN_5', 'HHN_6', 'HHN_7', 'geo_id']]
hs_census15['total_ppl'] = hs_census15['HHF_2']*2 + hs_census15['HHF_3']*3 + hs_census15['HHF_4']*4 + hs_census15['HHF_5']*5 + hs_census10['HHF_6']*6 + hs_census10['HHF_7']*7 + hs_census10['HHN_1'] + hs_census10['HHN_2']*2 + hs_census10['HHN_3']*3 + hs_census10['HHN_4']*4 + hs_census10['HHN_5']*5 + hs_census10['HHN_6']*6 + hs_census10['HHN_7']*7
hs_census15['avg_size'] = hs_census15['total_ppl']/ hs_census15['Total_HH']
hs_census_final15 = hs_census15[['Year', 'geo_id', 'avg_size']]

In [20]:
hs_census15_inf = hs_census15[hs_census15['avg_size'] == np.inf]

In [22]:
hs_census15_inf.head()

,Year,Total_HH,HHF_2,HHF_3,HHF_4,HHF_5,HHF_6,HHF_7,HHN_1,HHN_2,HHN_3,HHN_4,HHN_5,HHN_6,HHN_7,geo_id,total_ppl,avg_size
1691,2011-2015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120110106062,22,inf
2221,2011-2015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120110701011,167,inf
4447,2011-2015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120570065022,125,inf
5495,2011-2015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120710019072,89,inf
6846,2011-2015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,120860009032,124,inf


In [24]:
yearlist = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 
            2012, 2013, 2014, 2015, 2016]

In [25]:
hs_census_00 = hs_census_final[hs_census_final['Year'] == 2000]
hs_census_10 = hs_census_final[hs_census_final['Year'] == 2010]
dflist = []

for year in yearlist:
    if year < 2006:
        hs_census_00['Year'] = year
        dflist.append(hs_census_00.copy())
    elif year >= 2006 and year < 2010:
        hs_census_final10['Year'] = year
        dflist.append(hs_census_final10.copy())
    elif year == 2010:
        dflist.append(hs_census_10.copy())
    else:
        hs_census_final15['Year'] = year
        dflist.append(hs_census_final15.copy())
        
    
hs_final = pd.concat(dflist)

/Users/alenastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/alenastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/alenastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [29]:
# Replace infinite entries with 7

mask = hs_final.avg_size == np.inf
column_name = 'avg_size'
hs_final.loc[mask, column_name] = 7

In [33]:
# Replace values greater than 7 with 7
mask = hs_final.avg_size > 7
column_name = 'avg_size'
hs_final.loc[mask, column_name] = 7

In [35]:
hs_final.groupby('Year').count()

,geo_id,avg_size
Year,,
2000,74500,74143
2001,74500,74143
2002,74500,74143
2003,74500,74143
2004,74500,74143
2005,74500,74143
2006,74500,74006
2007,74500,74006
2008,74500,74006


In [34]:
hs_final.groupby('Year').max()

,geo_id,avg_size
Year,,
2000,551410117005,7.000000
2001,551410117005,7.000000
2002,551410117005,7.000000
2003,551410117005,7.000000
2004,551410117005,7.000000
2005,551410117005,7.000000
2006,551410117005,7.000000
2007,551410117005,7.000000
2008,551410117005,7.000000


In [37]:
hs_final.to_csv('raw/census/hs_final.csv', index = False)

In [39]:
hstest = pd.read_csv('raw/census/hs_final.csv')
hstest.head()

,Year,geo_id,avg_size
0,2000,100010401001,2.933535
1,2000,100010401002,2.846543
2,2000,100010401003,2.789916
3,2000,100010402011,2.600321
4,2000,100010402012,2.472630


In [40]:
hstest.groupby('Year').max()

,geo_id,avg_size
Year,,
2000,551410117005,7.000000
2001,551410117005,7.000000
2002,551410117005,7.000000
2003,551410117005,7.000000
2004,551410117005,7.000000
2005,551410117005,7.000000
2006,551410117005,7.000000
2007,551410117005,7.000000
2008,551410117005,7.000000


# Read and clean urban data

In [12]:
df = pd.read_csv('/Users/alenastern/Documents/Spring2018/Machine_Learning/evictions-learn/src/data/Urban_County_2010.csv', header = 0)

In [13]:
df.head()

,UA,UANAME,STATE,COUNTY,GEOID,CNAME,POPPT,HUPT,AREAPT,AREALANDPT,...,CAREA,CAREALAND,UAPOPPCT,UAHUPCT,UAAREAPCT,UAAREALANDPCT,CPOPPCT,CHUPCT,CAREAPCT,CAREALANDPCT
0,37,"Abbeville, LA Urban Cluster",22,45,22045,Iberia Parish,556,244,1304730.0,1304730.0,...,2.669056e+09,1.486940e+09,2.80,2.88,4.42,4.46,0.76,0.82,0.05,0.09
1,37,"Abbeville, LA Urban Cluster",22,113,22113,Vermilion Parish,19268,8216,28218638.0,27918141.0,...,3.993942e+09,3.038572e+09,97.20,97.12,95.58,95.54,33.22,32.56,0.71,0.92
2,64,"Abbeville, SC Urban Cluster",45,1,45001,Abbeville County,5243,2578,11334983.0,11315197.0,...,1.323464e+09,1.270348e+09,100.00,100.00,100.00,100.00,20.63,21.34,0.86,0.89
3,91,"Abbotsford, WI Urban Cluster",55,19,55019,Clark County,2863,1188,3274492.0,3261271.0,...,3.156643e+09,3.133407e+09,72.19,73.51,60.90,60.81,8.25,7.88,0.10,0.10
4,91,"Abbotsford, WI Urban Cluster",55,73,55073,Marathon County,1103,428,2102170.0,2102170.0,...,4.082627e+09,4.001488e+09,27.81,26.49,39.10,39.19,0.82,0.74,0.05,0.05


In [3]:
df = pd.read_csv('/Users/alenastern/Documents/Spring2018/Machine_Learning/evictions-learn/src/data/Urban_County_2010.csv', header = 0)
df = df[['UA', 'STATE', 'COUNTY', 'GEOID']]       

In [27]:
pct = df_states_urban.groupby('GEOID').sum()

In [28]:
pct_75 = pct[pct['CPOPPCT'] >= 75]

In [32]:
pct_75 = pct_75.reset_index()

In [33]:
df_75 = pct_75[['UA', 'STATE', 'COUNTY', 'GEOID']] 

In [34]:
df_75.shape

(213, 4)

In [22]:
df_states = df[(df['STATE'] == 18 ) | (df['STATE'] == 17) | (df['STATE'] == 26) | (df['STATE'] == 39) | (df['STATE'] == 55)
              | (df['STATE'] == 10) | (df['STATE'] == 11 ) | (df['STATE'] == 12) | (df['STATE'] == 13) | (df['STATE'] == 24)
              | (df['STATE'] == 37) | (df['STATE'] == 45) | (df['STATE'] == 51) | (df['STATE'] == 54)]

In [26]:
df_states_urban = df_states[df_states['UA'] != 99999]

In [35]:
df_75.to_csv('/Users/alenastern/Documents/Spring2018/Machine_Learning/evictions-learn/src/data/Urban_County_2010_states.csv', index = False)


In [9]:
df_states.shape

(2765, 4)

In [10]:
df_states_urban.shape

(1751, 4)